In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/My Drive/KnowledgeGraph/KG_Constraction/Model3"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive/


['entity-aware-relation-classification',
 'Model3_Run3_RE_BiLSTM.ipynb',
 'Model3_Run1.ipynb',
 'FewRel',
 'Model3_Run4_FewShot.ipynb',
 'Model3_Run2_Visualization.ipynb',
 'Model3_Run7_Visualization.ipynb',
 'Model3_Run5_DenpendencePath.ipynb',
 'Model3_Run9.ipynb',
 'exp_data.csv',
 'sentence_1.csv',
 'sentence_2.csv',
 'sentence_3.csv',
 'sentence_4.csv',
 'sentence_5.csv',
 'sentence_6.csv',
 'sentence_7.csv',
 'sentence_8.csv',
 'Model3_Run6_CSDN.ipynb',
 'Model3_Run8_OIE.ipynb',
 'Model3_Run6_Advanced_CSDN.ipynb']

In [2]:
!pip3 install spacy==2.1.0 # spacy最好使用2.1版本，否则会奇怪地报错
!python3 -m spacy download en_core_web_sm # 下载英文
!python3 -m spacy download zh_core_web_sm # 下载中文
!pip3 install neuralcoref==4.0.0 # 共指消解


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 27.7 MB 43.1 MB/s 
     |████████████████████████████████| 3.2 MB 49.9 MB/s 
     |████████████████████████████████| 2.1 MB 47.7 MB/s 
     |████████████████████████████████| 184 kB 50.8 MB/s 
     |████████████████████████████████| 82 kB 434 kB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.4
    Uninstalling srsly-2.4.4:
      Successfully uninstalled srsly-2.4.4
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.6
    Uninstalling preshed-3.0.6:
      Successfully uninstalled preshed-3.0.6
  Attempting uninstall: blis
    Found existing installation: blis 0.7.8
    Uninstalling blis-0.7.8:
      Successfully uninstalled blis-0.7.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting uni

In [3]:
import neuralcoref
import spacy
nlp = spacy.load("en_core_web_sm")
neuralcoref.add_to_pipe(nlp) # 将共指消解加入spacy的处理流程中
text = "My sister has a dog. She loves him"
doc = nlp(text) # 将text进行pipeline处理


100%|██████████| 40155833/40155833 [00:01<00:00, 25640647.27B/s]


In [4]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import re
import spacy
from spacy.attrs import intify_attrs
nlp = spacy.load("en_core_web_sm")

import neuralcoref

import networkx as nx
# import matplotlib.pyplot as plt

#nltk.download('stopwords')
from nltk.corpus import stopwords
all_stop_words = ['many', 'us', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',
                  'today', 'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
                  'september', 'october', 'november', 'december', 'today', 'old', 'new']
all_stop_words = sorted(list(set(all_stop_words + list(stopwords.words('english')))))

abspath = os.path.abspath('') ## String which contains absolute path to the script file
#print(abspath)
os.chdir(abspath)

### ==================================================================================================
# Tagger

def get_tags_spacy(nlp, text):
    doc = nlp(text) # 生成词对象
    entities_spacy = [] # Entities that Spacy NER found
    for ent in doc.ents: # doc.ents表示每个token的实体识别结果
        entities_spacy.append([ent.text, ent.start_char, ent.end_char, ent.label_])
    return entities_spacy

def tag_all(nlp, text, entities_spacy):
    if ('neuralcoref' in nlp.pipe_names):
        nlp.pipeline.remove('neuralcoref')    
    neuralcoref.add_to_pipe(nlp) # Add neural coref to SpaCy's pipe    
    doc = nlp(text)
    return doc

def filter_spans(spans):
    # Filter a sequence of spans so they don't contain overlaps
    get_sort_key = lambda span: (span.end - span.start, span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    return result

def tag_chunks(doc):
    spans = list(doc.ents) + list(doc.noun_chunks)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        string_store = doc.vocab.strings
        for span in spans:
            start = span.start
            end = span.end
            retokenizer.merge(doc[start: end], attrs=intify_attrs({'ent_type': 'ENTITY'}, string_store))

def tag_chunks_spans(doc, spans, ent_type):
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        string_store = doc.vocab.strings
        for span in spans:
            start = span.start
            end = span.end
            retokenizer.merge(doc[start: end], attrs=intify_attrs({'ent_type': ent_type}, string_store))

def clean(text):
    # 文本清理
    text = text.strip('[(),- :\'\"\n]\s*')
    text = text.replace('—', ' - ')
    text = re.sub('([A-Za-z0-9\)]{2,}\.)([A-Z]+[a-z]*)', r"\g<1> \g<2>", text, flags=re.UNICODE)
    text = re.sub('([A-Za-z0-9]{2,}\.)(\"\w+)', r"\g<1> \g<2>", text, flags=re.UNICODE)
    text = re.sub('([A-Za-z0-9]{2,}\.\/)(\w+)', r"\g<1> \g<2>", text, flags=re.UNICODE)
    text = re.sub('([[A-Z]{1}[[.]{1}[[A-Z]{1}[[.]{1}) ([[A-Z]{1}[a-z]{1,2} )', r"\g<1> . \g<2>", text, flags=re.UNICODE)
    text = re.sub('([A-Za-z]{3,}\.)([A-Z]+[a-z]+)', r"\g<1> \g<2>", text, flags=re.UNICODE)
    text = re.sub('([[A-Z]{1}[[.]{1}[[A-Z]{1}[[.]{1}) ([[A-Z]{1}[a-z]{1,2} )', r"\g<1> . \g<2>", text, flags=re.UNICODE)
    text = re.sub('([A-Za-z0-9]{2,}\.)([A-Za-z]+)', r"\g<1> \g<2>", text, flags=re.UNICODE)
    
    text = re.sub('’', "'", text, flags=re.UNICODE)           # curly apostrophe
    text = re.sub('‘', "'", text, flags=re.UNICODE)           # curly apostrophe
    text = re.sub('“', ' "', text, flags=re.UNICODE)
    text = re.sub('”', ' "', text, flags=re.UNICODE)
    text = re.sub("\|", ", ", text, flags=re.UNICODE)
    text = text.replace('\t', ' ')
    text = re.sub('…', '.', text, flags=re.UNICODE)           # elipsis
    text = re.sub('â€¦', '.', text, flags=re.UNICODE)          
    text = re.sub('â€“', '-', text)           # long hyphen
    text = re.sub('\s+', ' ', text, flags=re.UNICODE).strip()
    text = re.sub(' – ', ' . ', text, flags=re.UNICODE).strip()

    return text

def tagger(text):  
    df_out = pd.DataFrame(columns=['Document#', 'Sentence#', 'Word#', 'Word', 'EntityType', 'EntityIOB', 'Lemma', 'POS', 'POSTag', 'Start', 'End', 'Dependency'])
    corefs = [] # 保存所有指代的词
    text = clean(text) # 文本清理
    
    nlp = spacy.load("en_core_web_sm")
    entities_spacy = get_tags_spacy(nlp, text) # 获得每个token的实体识别结果
    #print("SPACY entities:\n", ([ent for ent in entities_spacy]), '\n\n')
    document = tag_all(nlp, text, entities_spacy) # 融入共指消解工具
    #for token in document:
        #print([token.i, token.text, token.ent_type_, token.ent_iob_, token.lemma_, token.pos_, token.tag_, token.idx, token.idx+len(token)-1, token.dep_])
    
    ### Coreferences
    # 
    if document._.has_coref:
        for cluster in document._.coref_clusters:
            main = cluster.main # 共指的词
            for m in cluster.mentions: # 所有指代的词（包括其本身）                   
                if (str(m).strip() == str(main).strip()): # 如果是其本身，则跳过
                    continue
                corefs.append([str(m), str(main)]) # 将所有指代的词加入corefs列表
    tag_chunks(document)    
    
    # chunk - somethin OF something 名词分块
    spans_change = []
    for i in range(2, len(document)):
        w_left = document[i-2]
        w_middle = document[i-1]
        w_right = document[i]
        if w_left.dep_ == 'attr':
            continue
        if w_left.ent_type_ == 'ENTITY' and w_right.ent_type_ == 'ENTITY' and (w_middle.text == 'of'): # or w_middle.text == 'for'): #  or w_middle.text == 'with'
            spans_change.append(document[w_left.i : w_right.i + 1])
    tag_chunks_spans(document, spans_change, 'ENTITY')
    
    # chunk verbs with multiple words: 'were exhibited' 动词分块
    spans_change_verbs = []
    for i in range(1, len(document)):
        w_left = document[i-1]
        w_right = document[i]
        if w_left.pos_ == 'VERB' and (w_right.pos_ == 'VERB'):
            spans_change_verbs.append(document[w_left.i : w_right.i + 1])
    tag_chunks_spans(document, spans_change_verbs, 'VERB')

    # chunk: verb + adp; verb + part 
    spans_change_verbs = []
    for i in range(1, len(document)):
        w_left = document[i-1]
        w_right = document[i]
        if w_left.pos_ == 'VERB' and (w_right.pos_ == 'ADP' or w_right.pos_ == 'PART'):
            spans_change_verbs.append(document[w_left.i : w_right.i + 1])
    tag_chunks_spans(document, spans_change_verbs, 'VERB')

    # chunk: adp + verb; part  + verb
    spans_change_verbs = []
    for i in range(1, len(document)):
        w_left = document[i-1]
        w_right = document[i]
        if w_right.pos_ == 'VERB' and (w_left.pos_ == 'ADP' or w_left.pos_ == 'PART'):
            spans_change_verbs.append(document[w_left.i : w_right.i + 1])
    tag_chunks_spans(document, spans_change_verbs, 'VERB')
    
    # chunk verbs with multiple words: 'were exhibited'
    spans_change_verbs = []
    for i in range(1, len(document)):
        w_left = document[i-1]
        w_right = document[i]
        if w_left.pos_ == 'VERB' and (w_right.pos_ == 'VERB'):
            spans_change_verbs.append(document[w_left.i : w_right.i + 1])
    tag_chunks_spans(document, spans_change_verbs, 'VERB')

    # chunk all between LRB- -RRB- (something between brackets)
    start = 0
    end = 0
    spans_between_brackets = []
    for i in range(0, len(document)):
        if ('-LRB-' == document[i].tag_ or r"(" in document[i].text):
            start = document[i].i
            continue
        if ('-RRB-' == document[i].tag_ or r')' in document[i].text):
            end = document[i].i + 1
        if (end > start and not start == 0):
            span = document[start:end]
            try:
                assert (u"(" in span.text and u")" in span.text)
            except:
                pass
                #print(span)
            spans_between_brackets.append(span)
            start = 0
            end = 0
    tag_chunks_spans(document, spans_between_brackets, 'ENTITY')
            
    # chunk entities  两个实体相邻时，合并
    spans_change_verbs = []
    for i in range(1, len(document)):
        w_left = document[i-1]
        w_right = document[i]
        if w_left.ent_type_ == 'ENTITY' and w_right.ent_type_ == 'ENTITY':
            spans_change_verbs.append(document[w_left.i : w_right.i + 1])
    tag_chunks_spans(document, spans_change_verbs, 'ENTITY')
    
    doc_id = 1
    count_sentences = 0
    prev_dep = 'nsubj'
    for token in document:
        if (token.dep_ == 'ROOT'):
            if token.pos_ == 'VERB':
                #  将pipeline的输出保存到csv，列名：['Document#', 'Sentence#', 'Word#', 'Word', 'EntityType', 'EntityIOB', 'Lemma', 'POS', 'POSTag', 'Start', 'End', 'Dependency']
                df_out.loc[len(df_out)] = [doc_id, count_sentences, token.i, token.text, token.ent_type_, token.ent_iob_, token.lemma_, token.pos_, token.tag_, token.idx, token.idx+len(token)-1, token.dep_]
            else:
                df_out.loc[len(df_out)] = [doc_id, count_sentences, token.i, token.text, token.ent_type_, token.ent_iob_, token.lemma_, token.pos_, token.tag_, token.idx, token.idx+len(token)-1, prev_dep]
        else:
            df_out.loc[len(df_out)] = [doc_id, count_sentences, token.i, token.text, token.ent_type_, token.ent_iob_, token.lemma_, token.pos_, token.tag_, token.idx, token.idx+len(token)-1, token.dep_]
                  
        if (token.text == '.'):
            count_sentences += 1
        prev_dep = token.dep_
        
    return df_out, corefs

### ==================================================================================================
### triple extractor

def get_predicate(s):
    pred_ids = {}
    for w, index, spo in s:
        if spo == 'predicate' and w != "'s" and w != "\"": #= 11.95
            pred_ids[index] = w
    predicates = {}
    for key, value in pred_ids.items():
        predicates[key] = value
    return predicates

def get_subjects(s, start, end, adps):
    subjects = {}
    for w, index, spo in s:
        if index >= start and index <= end:
            if 'subject' in spo or 'entity' in spo or 'object' in spo:
                subjects[index] = w
    return subjects
    
def get_objects(s, start, end, adps):
    objects = {}
    for w, index, spo in s:
        if index >= start and index <= end:
            if 'object' in spo or 'entity' in spo or 'subject' in spo:
                objects[index] = w
    return objects

def get_positions(s, start, end):
    adps = {}
    for w, index, spo in s:        
        if index >= start and index <= end:
            if 'of' == spo or 'at' == spo:
                adps[index] = w
    return adps

def create_triples(df_text, corefs):
    ## 创建三元组
    sentences = [] # 所有句子
    aSentence = [] # 某个句子
    
    for index, row in df_text.iterrows():  # 给每个词块打上标签 主要为四类
        d_id, s_id, word_id, word, ent, ent_iob, lemma, cg_pos, pos, start, end, dep = row.items()
        if 'subj' in dep[1]:
            aSentence.append([word[1], word_id[1], 'subject'])
        elif 'ROOT' in dep[1] or 'VERB' in cg_pos[1] or pos[1] == 'IN':
            aSentence.append([word[1], word_id[1], 'predicate'])
        elif 'obj' in dep[1]:
            aSentence.append([word[1], word_id[1], 'object'])
        elif ent[1] == 'ENTITY':
            aSentence.append([word[1], word_id[1], 'entity'])        
        elif word[1] == '.':
            sentences.append(aSentence)
            aSentence = []
        else:
            aSentence.append([word[1], word_id[1], pos[1]])
    
    relations = []
    #loose_entities = []
    for s in sentences:
        if len(s) == 0: continue
        preds = get_predicate(s) # 拿到一个句子里的所有标签为动词的单词块
        """
        if preds == {}: 
            preds = {p[1]:p[0] for p in s if (p[2] == 'JJ' or p[2] == 'IN' or p[2] == 'CC' or
                     p[2] == 'RP' or p[2] == ':' or p[2] == 'predicate' or
                     p[2] =='-LRB-' or p[2] =='-RRB-') }
            if preds == {}:
                #print('\npred = 0', s)
                preds = {p[1]:p[0] for p in s if (p[2] == ',')}
                if preds == {}:
                    ents = [e[0] for e in s if e[2] == 'entity']
                    if (ents):
                        loose_entities = ents # not significant for now
                        #print("Loose entities = ", ents)
        """
        if preds:
            if (len(preds) == 1):
                #print("preds = ", preds)
                predicate = list(preds.values())[0]
                if (len(predicate) < 2):
                    predicate = 'is'
                #print(s)
                ents = [e[0] for e in s if e[2] == 'entity']
                #print('ents = ', ents)
                for i in range(1, len(ents)):
                    relations.append([ents[0], predicate, ents[i]])

            pred_ids = list(preds.keys())
            pred_ids.append(s[0][1])
            pred_ids.append(s[len(s)-1][1])
            pred_ids.sort()
                    
            for i in range(1, len(pred_ids)-1):
                predicate = preds[pred_ids[i]]
                adps_subjs = get_positions(s, pred_ids[i-1], pred_ids[i]) 
                #print(adps_subjs)
                subjs = get_subjects(s, pred_ids[i-1], pred_ids[i], adps_subjs)  # 在两个predicate之间找后一个动词的主语
                adps_objs = get_positions(s, pred_ids[i], pred_ids[i+1])
                objs = get_objects(s, pred_ids[i], pred_ids[i+1], adps_objs) # 在两个predicate之间找前一个动词的宾语
                for k_s, subj in subjs.items():                
                    for k_o, obj in objs.items():
                        obj_prev_id = int(k_o) - 1
                        if obj_prev_id in adps_objs: # at, in, of
                            relations.append([subj, predicate + ' ' + adps_objs[obj_prev_id], obj])
                        else:
                            relations.append([subj, predicate, obj])
                print(predicate,relations)
    
    ### Read coreferences: coreference files are TAB separated values
    coreferences = []
    for val in corefs:
        if val[0].strip() != val[1].strip():
            if len(val[0]) <= 50 and len(val[1]) <= 50:
                co_word = val[0]
                real_word = val[1].strip('[,- \'\n]*')
                real_word = re.sub("'s$", '', real_word, flags=re.UNICODE)
                if (co_word != real_word):
                    coreferences.append([co_word, real_word])
            else:
                co_word = val[0]
                real_word = ' '.join((val[1].strip('[,- \'\n]*')).split()[:7])
                real_word = re.sub("'s$", '', real_word, flags=re.UNICODE)
                if (co_word != real_word):
                    coreferences.append([co_word, real_word])
                
    # Resolve corefs
    triples_object_coref_resolved = []
    triples_all_coref_resolved = []
    for s, p, o in relations:
        coref_resolved = False
        for co in coreferences:
            if (s == co[0]):
                subj = co[1]
                triples_object_coref_resolved.append([subj, p, o])
                coref_resolved = True
                break
        if not coref_resolved:
            triples_object_coref_resolved.append([s, p, o])

    for s, p, o in triples_object_coref_resolved:
        coref_resolved = False
        for co in coreferences:
            if (o == co[0]):
                obj = co[1]
                triples_all_coref_resolved.append([s, p, obj])
                coref_resolved = True
                break
        if not coref_resolved:
            triples_all_coref_resolved.append([s, p, o])
    return(triples_all_coref_resolved)

def get_graph(triples):
    G = nx.DiGraph()
    for s, p, o in triples:
        G.add_edge(s, o, key=p)
    return G

def get_entities_with_capitals(G):
    entities = []
    for node in G.nodes():
        if (any(ch.isupper() for ch in list(node))):
            entities.append(node)
    return entities

def get_paths_between_capitalised_entities(triples):
    
    g = get_graph(triples)
    ents_capitals = get_entities_with_capitals(g)
    print(ents_capitals)
    paths = []
    #print('\nShortest paths among capitalised words -------------------')
    for i in range(0, len(ents_capitals)):
        n1 = ents_capitals[i]
        for j in range(1, len(ents_capitals)):
            try:
                n2 = ents_capitals[j]
                path = nx.shortest_path(g, source=n1, target=n2)
                if path and len(path) > 2:
                    paths.append(path)
                path = nx.shortest_path(g, source=n2, target=n1)
                if path and len(path) > 2:
                    paths.append(path)
            except Exception:
                continue
    return g, paths


def get_paths(doc_triples):
    triples = []
    g, paths = get_paths_between_capitalised_entities(doc_triples)
    for p in paths:
        path = [(u, g[u][v]['key'], v) for (u, v) in zip(p[0:], p[1:])]
        length = len(p)
        if (path[length-2][1] == 'in' or path[length-2][1] == 'at' or path[length-2][1] == 'on'):
            if [path[0][0], path[length-2][1], path[length-2][2]] not in triples:
                triples.append([path[0][0], path[length-2][1], path[length-2][2]])
        elif (' in' in path[length-2][1] or ' at' in path[length-2][1] or ' on' in path[length-2][1]):
            if [path[0][0], path[length-2][1], path[length-2][2]] not in triples:
                triples.append([path[0][0], 'in', path[length-2][2]])
    for t in doc_triples:
        if t not in triples:
            triples.append(t)
    return triples


# 抽取三元组
def extract_triples(text):
    df_tagged, corefs = tagger(text) # pipeline处理文本，并返回每个token的特征，以及共指消解的结果
    doc_triples = create_triples(df_tagged, corefs)
    all_triples = get_paths(doc_triples)
    filtered_triples = []    
    for s, p, o in all_triples:
        if ([s, p, o] not in filtered_triples):
            if s.lower() in all_stop_words or o.lower() in all_stop_words:
                continue
            elif s == p:
                continue
            if s.isdigit() or o.isdigit():
                continue
            if '%' in o or '%' in s: #= 11.96
                continue
            if (len(s) < 2) or (len(o) < 2):
                continue
            if (s.islower() and len(s) < 4) or (o.islower() and len(o) < 4):
                continue
            if s == o:
                continue            
            subj = s.strip('[,- :\'\"\n]*')
            pred = p.strip('[- :\'\"\n]*.')
            obj = o.strip('[,- :\'\"\n]*')
            
            for sw in ['a', 'an', 'the', 'its', 'their', 'his', 'her', 'our', 'all', 'old', 'new', 'latest', 'who', 'that', 'this', 'these', 'those']:
                subj = ' '.join(word for word in subj.split() if not word == sw)
                obj = ' '.join(word for word in obj.split()  if not word == sw)
            subj = re.sub("\s\s+", " ", subj)
            obj = re.sub("\s\s+", " ", obj)
            
            if subj and pred and obj:
                filtered_triples.append([subj, pred, obj])

    #TRIPLES = rank_by_degree(filtered_triples)
    return filtered_triples

In [6]:
sentence_1 = 'with the significate progress in natural language processing and computer vision, extracting valuable information from a lot of bio medical literature has gained great popularity among researchers ,and deep learning has boosted the development of effective biomedical text mining models.'
sentence_2 = 'While BERT obtains performance comparable to that of previous models, BioBERT significantly outperforms them on the following three representative biomedical text mining tasks: biomedical named entity recognition, biomedical relation extraction and biomedical question answering.'
sentence_3 = 'We view this problem as a link prediction task in a knowledge graph, and show that a new model of the top discriminative meta paths is able to understand the meaning of some statement and accurately determine its veracity.'
sentence_4 = 'the story is that cui win the game.'
sentence_5 = 'We evaluate our approach by examining thousands of claims related to history, geography, biology, and politics using a public million node knowledge graph extracted from Wikipedia and PubMedDB.'
sentence_6 = 'Biomedical text mining is becoming increasingly important as the number of biomedical documents rapidly grows.'  # 这种没有主谓宾的结构是抽不出来的
sentence_7 = 'However, directly applying the advancements in NLP to biomedical text mining often yields unsatisfactory results due to a word distribution shift from general domain corpora to biomedical corpora.'
sentence_8 = 'With almost the same architecture across tasks, BioBERT largely outperforms BERT and previous models in a variety of biomedical text mining tasks when pre-trained on biomedical corpora.'

df_tagged, corefs = tagger(sentence_8)
df_tagged.to_csv("sentence_8.csv",index=False)
df_tagged

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: FutureWarning: Possible nested set at position 2
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: FutureWarning: Possible nested set at position 11
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: FutureWarning: Possible nested set at position 18
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: FutureWarning: Possible nested set at position 27
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:80: FutureWarning: Possible nested set at position 37


,Document#,Sentence#,Word#,Word,EntityType,EntityIOB,Lemma,POS,POSTag,Start,End,Dependency
0,1,0,0,With,,O,with,ADP,IN,0,3,prep
1,1,0,1,almost the same architecture,ENTITY,O,almost,ADV,RB,5,32,pobj
2,1,0,2,across,,O,across,ADP,IN,34,39,prep
3,1,0,3,tasks,ENTITY,O,task,NOUN,NNS,41,45,pobj
4,1,0,4,",",,O,",",PUNCT,",",46,46,punct
5,1,0,5,BioBERT,ENTITY,O,BioBERT,NOUN,NNS,48,54,nsubj
6,1,0,6,largely,,O,largely,ADV,RB,56,62,advmod
7,1,0,7,outperforms,,O,outperform,VERB,VBZ,64,74,ROOT
8,1,0,8,BERT,,O,bert,ADJ,JJ,76,79,dobj
9,1,0,9,and,,O,and,CCONJ,CC,81,83,cc


In [9]:
sentence_9 = 'We expand the item response theory to study the case of cheating students for a set of exams , trying to detect cheating students by applying a greedy algorithm of inference . a greedy algorithm of inference is closely related to the Boltzmann machine learning .'
df_tagged, corefs = tagger(sentence_9)
df_tagged.to_csv("sentence_9.csv",index=False)
df_tagged

,Document#,Sentence#,Word#,Word,EntityType,EntityIOB,Lemma,POS,POSTag,Start,End,Dependency
0,1,0,0,We,ENTITY,O,-PRON-,PRON,PRP,0,1,nsubj
1,1,0,1,expand,,O,expand,VERB,VBP,3,8,ROOT
2,1,0,2,the item response theory,ENTITY,O,the,DET,DT,10,33,dobj
3,1,0,3,to study,VERB,O,to,PART,TO,35,42,advcl
4,1,0,4,the case,ENTITY,O,the,DET,DT,44,51,dobj
5,1,0,5,of cheating,VERB,O,of,ADP,IN,53,63,prep
6,1,0,6,students,ENTITY,O,student,NOUN,NNS,65,72,dobj
7,1,0,7,for,,O,for,ADP,IN,74,76,prep
8,1,0,8,a set of exams,ENTITY,O,a,DET,DT,78,91,pobj
9,1,0,9,",",,O,",",PUNCT,",",93,93,punct


In [ ]:
from spacy import displacy
doc = nlp(sentence_1)
displacy.render(doc, style='dep', jupyter=True,options = {'distance': 120})

NameError: ignored

In [ ]:
df_tagged, corefs = tagger(text)
doc_triples = create_triples(df_tagged, corefs)
all_triples = get_paths(doc_triples)

compare [['We', 'compare', 'a standard approach'], ['We', 'compare', 'the sparse interactions']]
in [['We', 'compare', 'a standard approach'], ['We', 'compare', 'the sparse interactions'], ['a standard approach', 'in', 'the Boltzmann machine'], ['the sparse interactions', 'in', 'the Boltzmann machine']]
learning to [['We', 'compare', 'a standard approach'], ['We', 'compare', 'the sparse interactions'], ['a standard approach', 'in', 'the Boltzmann machine'], ['the sparse interactions', 'in', 'the Boltzmann machine'], ['the Boltzmann machine', 'learning to', 'our greedy algorithm'], ['the Boltzmann machine', 'learning to', 'we']]
find [['We', 'compare', 'a standard approach'], ['We', 'compare', 'the sparse interactions'], ['a standard approach', 'in', 'the Boltzmann machine'], ['the sparse interactions', 'in', 'the Boltzmann machine'], ['the Boltzmann machine', 'learning to', 'our greedy algorithm'], ['the Boltzmann machine', 'learning to', 'we'], ['our greedy algorithm', 'find', 'a gree

In [ ]:
doc_triples

[['We', 'compare', 'a standard approach'],
 ['We', 'compare', 'the sparse interactions'],
 ['a standard approach', 'in', 'the Boltzmann machine'],
 ['the sparse interactions', 'in', 'the Boltzmann machine'],
 ['the Boltzmann machine', 'learning to', 'our greedy algorithm'],
 ['the Boltzmann machine', 'learning to', 'we'],
 ['our greedy algorithm', 'find', 'a greedy algorithm of inference'],
 ['we', 'find', 'a greedy algorithm of inference'],
 ['a greedy algorithm of inference', 'in', 'several aspects']]

In [ ]:
all_triples

[['We', 'in', 'the Boltzmann machine'],
 ['We', 'compare', 'a standard approach'],
 ['We', 'compare', 'the sparse interactions'],
 ['a standard approach', 'in', 'the Boltzmann machine'],
 ['the sparse interactions', 'in', 'the Boltzmann machine'],
 ['the Boltzmann machine', 'learning to', 'our greedy algorithm'],
 ['the Boltzmann machine', 'learning to', 'we'],
 ['our greedy algorithm', 'find', 'a greedy algorithm of inference'],
 ['we', 'find', 'a greedy algorithm of inference'],
 ['a greedy algorithm of inference', 'in', 'several aspects']]

In [ ]:
df_tagged

,Document#,Sentence#,Word#,Word,EntityType,EntityIOB,Lemma,POS,POSTag,Start,End,Dependency
0,1,0,0,We,ENTITY,O,-PRON-,PRON,PRP,0,1,nsubj
1,1,0,1,compare,,O,compare,VERB,VBP,3,9,ROOT
2,1,0,2,a standard approach,ENTITY,O,a,DET,DT,11,29,dobj
3,1,0,3,to infer,VERB,O,to,PART,TO,31,38,advcl
4,1,0,4,the sparse interactions,ENTITY,O,the,DET,DT,40,62,dobj
5,1,0,5,in,,O,in,ADP,IN,64,65,prep
6,1,0,6,the Boltzmann machine,ENTITY,O,the,DET,DT,67,87,pobj
7,1,0,7,learning to,VERB,O,learn,VERB,VBG,89,99,acl
8,1,0,8,our greedy algorithm,ENTITY,O,-PRON-,DET,PRP$,101,120,pobj
9,1,0,9,and,,O,and,CCONJ,CC,122,124,cc


In [ ]:
### ==================================================================================================
## Get more using Network shortest_paths


def get_paths(doc_triples):
    triples = []
    g, paths = get_paths_between_capitalised_entities(doc_triples)
    for p in paths:
        path = [(u, g[u][v]['key'], v) for (u, v) in zip(p[0:], p[1:])]
        length = len(p)
        if (path[length-2][1] == 'in' or path[length-2][1] == 'at' or path[length-2][1] == 'on'):
            if [path[0][0], path[length-2][1], path[length-2][2]] not in triples:
                triples.append([path[0][0], path[length-2][1], path[length-2][2]])
        elif (' in' in path[length-2][1] or ' at' in path[length-2][1] or ' on' in path[length-2][1]):
            if [path[0][0], path[length-2][1], path[length-2][2]] not in triples:
                triples.append([path[0][0], 'in', path[length-2][2]])
    for t in doc_triples:
        if t not in triples:
            triples.append(t)
    return triples

def get_center(nodes):
    center = ''
    if (len(nodes) == 1):
        center = nodes[0]
    else:   
        # Capital letters and longer is preferred
        cap_ents = [e for e in nodes if any(x.isupper() for x in e)]
        if (cap_ents):
            center = max(cap_ents, key=len)
        else:
            center = max(nodes, key=len)
    return center

def connect_graphs(mytriples):
    G = nx.DiGraph()
    for s, p, o in mytriples:
        G.add_edge(s, o, p=p)        
    
    """
    # Get components
    graphs = list(nx.connected_component_subgraphs(G.to_undirected()))
    
    # Get the largest component
    largest_g = max(graphs, key=len)
    largest_graph_center = ''
    largest_graph_center = get_center(nx.center(largest_g))
    
    # for each graph, find the centre node
    smaller_graph_centers = []
    for g in graphs:        
        center = get_center(nx.center(g))
        smaller_graph_centers.append(center)

    for n in smaller_graph_centers:
        if (largest_graph_center is not n):
            G.add_edge(largest_graph_center, n, p='with')
    """
    return G
        
def rank_by_degree(mytriples): #, limit):
    G = connect_graphs(mytriples)
    degree_dict = dict(G.degree(G.nodes()))
    nx.set_node_attributes(G, degree_dict, 'degree')
    
    # Use this to draw the graph
    #draw_graph_centrality(G, degree_dict)

    Egos = nx.DiGraph()
    for a, data in sorted(G.nodes(data=True), key=lambda x: x[1]['degree'], reverse=True):
        ego = nx.ego_graph(G, a)
        Egos.add_edges_from(ego.edges(data=True))
        Egos.add_nodes_from(ego.nodes(data=True))
        
        #if (nx.number_of_edges(Egos) > 20):
        #    break
       
    ranked_triples = []
    for u, v, d in Egos.edges(data=True):
        ranked_triples.append([u, d['p'], v])
    return ranked_triples


def draw_graph_centrality(G, dictionary):
    # plt.figure(figsize=(12,10))
    # pos = nx.spring_layout(G)
    # #print("Nodes\n", G.nodes(True))
    # #print("Edges\n", G.edges())
    
    # nx.draw_networkx_nodes(G, pos, 
    #         nodelist=dictionary.keys(),
    #         with_labels=False,
    #         edge_color='black',
    #         width=1,
    #         linewidths=1,
    #         node_size = [v * 150 for v in dictionary.values()],
    #         node_color='blue',
    #         alpha=0.5)
    # edge_labels = {(u, v): d["p"] for u, v, d in G.edges(data=True)}
    # #print(edge_labels)
    # nx.draw_networkx_edge_labels(G, pos,
    #                        font_size=10,
    #                        edge_labels=edge_labels,
    #                        font_color='blue')
    # nx.draw(G, pos, with_labels=True, node_size=1, node_color='blue')
    pass

In [35]:
import re

sentence ='[ARG0: Bob] agreed to (V: take) out (ARG1: the trash)'
#result_list = re.findall(r"[[](.*?)[]]", sentence)

#print(result_list [0])


In [21]:
sentence[11:]

' agreed to [V: take] out (ARG1: the trash)'

In [29]:
re.findall(r"[(](.*?)[)]", sentence)

['ARG0: Bob', 'V: take', 'ARG1: the trash']

In [37]:
re.sub('\[','(',sentence)

'(ARG0: Bob] agreed to (V: take) out (ARG1: the trash)'